In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
LEARNED_THR = 0.00001
IOU_THR = 0.7
PREDICTION_PATH = "dataset/predictions/09-predictions.csv"

In [3]:
PREDICTION_IMAGE_PATH = Path("dataset/images/test")

test_df = pd.read_csv(PREDICTION_PATH).rename(
    columns={"x_min": "xmin", "y_min": "ymin", "x_max": "xmax", "y_max": "ymax"}
)
test_df.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
28262,ID_AVXfJI.jpg,cssvd,0.000046,5.341148,3.300514,489.491821,327.946442
20576,ID_xRuM49.jpg,healthy,0.001925,0.000000,523.858765,125.888214,950.162476
14909,ID_eQxB51.jpg,healthy,0.000010,311.175903,810.000000,355.965118,810.000000
67097,ID_UbtyYv.jpg,cssvd,0.000035,1045.279907,231.740479,1079.963867,421.496948
198941,ID_Q1sJBq.jpg,anthracnose,0.000041,1239.282593,486.060181,1279.770386,906.444214


In [4]:
class_id = {j: i for i, j in enumerate(sorted(test_df["class"].unique()))}
test_df["class_id"] = test_df["class"].map(class_id)
class_id

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [5]:
test_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,healthy,0.752511,66.324226,0.000000,3809.420410,1697.234009,2
1,ID_cWEAQI.jpeg,healthy,0.064991,338.524902,1380.339478,1116.079102,1796.825317,2
2,ID_cWEAQI.jpeg,anthracnose,0.016846,2950.380615,606.978638,3975.202393,1475.716309,0
3,ID_cWEAQI.jpeg,healthy,0.005849,168.384781,0.000000,1498.166260,472.348175,2
4,ID_cWEAQI.jpeg,anthracnose,0.002263,3206.231445,821.201416,3986.431396,1348.242676,0


In [6]:
import torch

def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}


converted_labels = list(PREDICTION_IMAGE_PATH.glob("*"))
converted_labels = [i.name for i in converted_labels]

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	print(len(preds))
	print(list(preds.keys())[:10])
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [7]:
test_df.sample(2)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
98540,ID_vfelPl.jpg,anthracnose,0.000012,0.121983,1227.896851,143.486084,1750.469360,0
101074,ID_jr7JD5.jpg,anthracnose,0.000006,846.668518,435.631470,1418.477539,873.159302,0


In [8]:
predictions = get_preds_data(test_df, thr=None)

len(predictions)

1626
['ID_A16nzu.jpg', 'ID_A1Euyz.jpg', 'ID_A1HcV0.jpeg', 'ID_A4ZdJC.jpeg', 'ID_A5SFUW.jpeg', 'ID_A6Fogi.jpeg', 'ID_ABDCyn.jpeg', 'ID_ACg6Qf.jpeg', 'ID_AFi8A1.jpg', 'ID_AHlc9P.jpg']


1626

In [9]:
# predictions["ID_CGnAYP.jpg"]

In [10]:
predictions["ID_CGnAYP.jpg"].keys()

dict_keys(['boxes', 'scores', 'labels'])

In [11]:
from torchvision.ops import nms

def apply_nms(predictions: dict[str, torch.Tensor], iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression (NMS) to reduce overlapping boxes.

    Args:
        predictions (dict): Dictionary containing keys ['boxes', 'scores', 'labels'].
        iou_threshold (float): Intersection over Union (IoU) threshold for NMS.

    Returns:
        dict: Filtered predictions after applying NMS.
    """
    filtered_predictions = {}
    for image_id, data in predictions.items():
        boxes = data["boxes"]
        scores = data["scores"]
        labels = data["labels"]

        if boxes.numel() == 0:
            # If no boxes, skip this image
            filtered_predictions[image_id] = data
            continue

        # Perform NMS for each class separately
        keep_indices = []
        for label in labels.unique():
            label_mask = labels == label
            label_boxes = boxes[label_mask]
            label_scores = scores[label_mask]
            indices = nms(label_boxes, label_scores, iou_threshold)
            keep_indices.extend(label_mask.nonzero(as_tuple=True)[0][indices].tolist())

        # Filter boxes, scores, and labels based on NMS results
        keep_indices = torch.tensor(keep_indices, dtype=torch.long)
        filtered_predictions[image_id] = {
            "boxes": boxes[keep_indices],
            "scores": scores[keep_indices],
            "labels": labels[keep_indices],
        }

    return filtered_predictions

# Apply NMS to predictions
filtered_predictions = apply_nms(predictions, iou_threshold=IOU_THR)

In [12]:
# for k, v in predictions.items():
#     r = filtered_predictions[k]
#     if len(v["boxes"]) != len(r["boxes"]):
#         print(k, len(v["boxes"]), len(r["boxes"]))
#         print("Boxes before NMS:", v["boxes"])
#         print("Boxes after NMS:", r["boxes"])
#         print("Scores before NMS:", v["scores"])
#         print("Scores after NMS:", r["scores"])
#         print("Labels before NMS:", v["labels"])
#         print("Labels after NMS:", r["labels"])
#         print()
#         break

In [13]:
filtered_data = []
id_to_label = {v: k for k, v in class_id.items()}
for image_id, data in filtered_predictions.items():
    boxes = data["boxes"].tolist()
    scores = data["scores"].tolist()
    labels = data["labels"].tolist()
    for box, score, label in zip(boxes, scores, labels):
        filtered_data.append({
            "Image_ID": image_id,
            "class": id_to_label[label],
            "confidence": score,
            "ymin": box[1],
            "xmin": box[0],
            "ymax": box[3],
            "xmax": box[2],
            "class_id": label
        })

filtered_df = pd.DataFrame(filtered_data)
filtered_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,anthracnose,0.016846,2950.380615,606.978638,3975.202393,1475.716309,0
1,ID_cWEAQI.jpeg,anthracnose,0.002263,3206.231445,821.201416,3986.431396,1348.242676,0
2,ID_cWEAQI.jpeg,anthracnose,0.001027,3613.622803,0.000000,4000.000000,720.362122,0
3,ID_cWEAQI.jpeg,anthracnose,0.000755,2752.107910,51.489349,3995.725830,1658.611694,0
4,ID_cWEAQI.jpeg,anthracnose,0.000599,2968.642090,869.716614,3623.279297,1310.144775,0


In [14]:
len(filtered_df), len(test_df)

(242592, 243900)

In [15]:
filtered_df["class"].value_counts()

class
healthy        102924
cssvd           75439
anthracnose     64229
Name: count, dtype: int64

In [16]:
filtered_df["class"].value_counts(normalize=True)

class
healthy        0.424268
cssvd          0.310971
anthracnose    0.264761
Name: proportion, dtype: float64

In [17]:
files = list(PREDICTION_IMAGE_PATH.glob("*"))

In [18]:
test_df: pd.DataFrame = filtered_df.copy()

In [19]:
test_records = test_df.to_dict(orient="records")

test_records[:5]

[{'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.016845766454935074,
  'ymin': 2950.380615234375,
  'xmin': 606.9786376953125,
  'ymax': 3975.202392578125,
  'xmax': 1475.71630859375,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0022628314327448606,
  'ymin': 3206.2314453125,
  'xmin': 821.201416015625,
  'ymax': 3986.431396484375,
  'xmax': 1348.24267578125,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0010270517086610198,
  'ymin': 3613.622802734375,
  'xmin': 0.0,
  'ymax': 4000.0,
  'xmax': 720.3621215820312,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0007548005087301135,
  'ymin': 2752.10791015625,
  'xmin': 51.489349365234375,
  'ymax': 3995.725830078125,
  'xmax': 1658.6116943359375,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0005994461826048791,
  'ymin':

In [20]:
file_id = [i.name for i in files]
file_id[:2]

['ID_cWEAQI.jpeg', 'ID_NtqErb.jpg']

In [21]:
file_okay = set(test_df["Image_ID"].values)
file_nokay = set(file_id) - file_okay

for f in file_nokay:
    test_records.append(
		{
			"Image_ID": f,
		}
	)

In [22]:
test_df = pd.DataFrame(test_records)

test_df.sample(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
89994,ID_Ivijgf.jpg,anthracnose,0.000108,0.000000,344.714417,30.341446,603.604675,0
197292,ID_AuxMUi.jpeg,healthy,0.000137,2778.057129,3024.000000,2975.129883,3024.000000,2
5125,ID_MISf6G.jpeg,anthracnose,0.000012,923.500000,805.786194,1420.819702,1358.097778,0
94375,ID_VLFJSv.jpg,anthracnose,0.000020,870.469849,0.000000,1262.002319,62.525909,0
236539,ID_GMWGve.jpg,cssvd,0.000007,1016.283508,568.519165,1079.885742,781.297852,1
18826,ID_JgQ193.jpg,anthracnose,0.000015,585.394287,122.200867,701.264160,230.452347,0
198479,ID_zxvYPO.jpeg,anthracnose,0.000009,0.000000,975.839905,358.819519,2870.478271,0
12308,ID_V7syTo.jpg,anthracnose,0.000039,1.651108,19.337585,225.556229,319.822632,0
101077,ID_ft52kR.jpg,cssvd,0.000028,16.761703,249.625092,576.891846,624.434326,1
218660,ID_EmeUOz.jpg,healthy,0.000326,20.472578,93.852570,61.553406,132.014084,2


In [23]:
test_df["class"].value_counts()

class
healthy        102924
cssvd           75439
anthracnose     64229
Name: count, dtype: int64

In [24]:
test_df.to_csv(PREDICTION_PATH.replace("-prediction", "-submission"), index=False)

In [25]:
PREDICTION_PATH.replace("-prediction", "-submission")

'dataset/predictions/09-submissions.csv'